# Hope this one works

In [62]:
import os, json, inspect
from dotenv import load_dotenv
from typing import Callable, List

from mistralai import Mistral, UserMessage, ToolMessage

load_dotenv(override=True)

API_KEY = os.getenv("MISTRAL_API_KEY")
if not API_KEY:
    print("Warning: MISTRAL_API_KEY is not set.")

MODEL_NAME = os.getenv("MISTRAL_MODEL", "mistral-large-latest")
TEMPERATURE = float(os.getenv("MISTRAL_TEMPERATURE", "0.0"))
client = Mistral(api_key=API_KEY)

print(f"Environment loaded! Using model: {MODEL_NAME}")

def build_tool_spec(func: Callable):
    """Build a tool spec dict from a plain python function."""
    sig = inspect.signature(func)
    props = {}
    required = []
    for name, param in sig.parameters.items():
        ann = param.annotation
        ann_type = "string"
        if ann in (int, float):
            ann_type = "number"
        props[name] = {"type": ann_type}
        if param.default is inspect._empty:
            required.append(name)
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip()[:800],
            "parameters": {
                "type": "object",
                "properties": props,
                "required": required
            }
        }
    }

def run_tool_chat(user_content: str, funcs: List[Callable], model: str = MODEL_NAME, temperature: float = TEMPERATURE):
    """Send a user message, handle any tool calls, return final answer string."""
    messages = [UserMessage(role="user", content=user_content)]
    tool_specs = [build_tool_spec(f) for f in funcs]
    first = client.chat.complete(model=model, messages=messages, tools=tool_specs, temperature=temperature)
    msg = first.choices[0].message
    tool_calls = msg.tool_calls or []
    if not tool_calls:
        return msg.content
    messages.append(msg)
    for tc in tool_calls:
        args = json.loads(tc.function.arguments)
        fn = next((f for f in funcs if f.__name__ == tc.function.name), None)
        if fn is None:
            result = f"Error: function {tc.function.name} not implemented"
        else:
            try:
                result = fn(**args)
            except Exception as e:
                result = f"Error executing {tc.function.name}: {e}".strip()
        print(f"Tool {tc.function.name}({args}) -> {str(result)[:160]}")
        messages.append(ToolMessage(role="tool", content=str(result), name=tc.function.name, tool_call_id=tc.id))
    final = client.chat.complete(model=model, messages=messages, temperature=temperature)
    return final.choices[0].message.content

Environment loaded! Using model: mistral-large-latest


In [48]:
def get_ticketmaster_events(classificationName: str) -> str:
    """Fetch music/sports events from Ticketmaster API in Belgium.
    
    This returns events from the saved JSON file matching the classification 
    formatted with all details. The caller can filter this list to get the best N results.
    
    Args:
        classificationName (str): Event type filter: 'Music', 'Sports', 'Arts', 'Film'.
    
    Returns:
        str: Formatted list of events with name, date, venue, address, price, and link.
    """
    
    # Map classification to JSON file
    file_map = {
        'music': 'ApiData8_12_25/ticketmaster_events_Music.json',
        'sports': 'ApiData8_12_25/ticketmaster_events_Sports.json',
        'arts': 'ApiData8_12_25/ticketmaster_events_Arts.json',
        'film': 'ApiData8_12_25/ticketmaster_events_Film.json'
    }
    
    file_path = file_map.get(classificationName.lower())
    if not file_path:
        return f"Error: Unknown classification '{classificationName}'. Use: Music, Sports, Arts, or Film"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except FileNotFoundError:
        return f"Error: File not found: {file_path}"
    
    result = []
    for i, event in enumerate(events, 1):
        title = event.get('title', 'N/A')
        date = event.get('date', 'N/A')
        start_time = event.get('start_time', '')
        where = event.get('where', 'N/A')
        address = event.get('address', 'N/A')
        price = event.get('price', {})
        link = event.get('link', 'N/A')
        
        if isinstance(price, dict):
            price_str = f"{price.get('min', '?')} - {price.get('max', '?')} {price.get('currency', 'EUR')}"
        else:
            price_str = str(price)
        
        result.append(f"{i}. {title}\n   📅 {date} {start_time}\n   📍 {where} - {address}\n   💰 {price_str}\n   🔗 {link}\n")
    
    return "\n".join(result) if result else "No events found"

In [49]:
def get_brussels_events(category: str) -> str:
    """Fetch events from Brussels API in French.
    
    This returns events from the saved JSON file for the specified category 
    formatted with all details. The caller can filter this list to get the best N results.
    
    Args:
        category (str): Event category: 'Concert', 'Show', 'Exhibition', 'Theatre', 'Clubbing', 'Cinema', 'Sport'.
    
    Returns:
        str: Formatted list of events with name, date, venue, address, price, and description.
    """
    
    # Map category to JSON file
    file_map = {
        'concert': 'ApiData8_12_25/brussels_events_Concert.json',
        'show': 'ApiData8_12_25/brussels_events_Show.json',
        'exhibition': 'ApiData8_12_25/brussels_events_Exhibition.json',
        'theatre': 'ApiData8_12_25/brussels_events_Theatre.json',
        'clubbing': 'ApiData8_12_25/brussels_events_Clubbing.json',
        'cinema': 'ApiData8_12_25/brussels_events_Cinema.json',
        'sport': 'ApiData8_12_25/brussels_events_Sport.json'
    }
    
    file_path = file_map.get(category.lower())
    if not file_path:
        return f"Error: Unknown category '{category}'. Use: Concert, Show, Exhibition, Theatre, Clubbing, Cinema, or Sport"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except FileNotFoundError:
        return f"Error: File not found: {file_path}"
    
    result = []
    for i, event in enumerate(events, 1):
        title = event.get('title', 'N/A')
        date_start = event.get('date_start', 'N/A')
        date_end = event.get('date_end', '')
        venue = event.get('where', 'N/A')
        address = event.get('address', 'N/A')
        description = event.get('description', 'N/A')
        price = event.get('price', 'N/A')
        link = event.get('link', 'N/A')




        result.append(f"{i}. {title}\n   📅 Du {date_start} au {date_end}\n   📍 {venue} - {address}\n   📝 {description}\n   💰 {price}\n   🔗 {link}\n")
    print(result)




        
    
    return "\n".join(result) if result else "No events found"

In [50]:
def get_eventBrite_events() -> str:
    """Get upcoming events from EventBrite.
    
    This returns events from the saved EventBrite JSON file with all details.
    The caller can filter this list to get the best events.
    
    Returns:
        str: Formatted list of events with name, date, time, venue, address, and link.
    """
    
    file_path = 'ApiData8_12_25/eventbrite_events_ALL.json'
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            events = json.load(f)
    except FileNotFoundError:
        return f"Error: File not found: {file_path}"
    
    result = []
    for i, event in enumerate(events, 1):
        title = event.get('name', 'N/A')
        date = event.get('start', {}).get('local', 'N/A')
        end_time = event.get('end', {}).get('local', '')
        where = event.get('where', 'N/A')
        address = event.get('address', 'N/A')
        description = event.get('description', '')
        link = event.get('url', 'N/A')
        
        result.append(f"{i}. {title}\n   📅 {date} {end_time}\n   📍 {where}\n   Adresse: {address}\n   📝 {description}\n   🔗 {link}\n")
    
    return "\n".join(result) if result else "No events found"

In [51]:
# Example: Use with run_tool_chat
print("=== Events from JSON files via Mistral Tool Calling ===\n")

answer = run_tool_chat(
    "Trouve moi EXACTEMENT 5 evenements musicaux à Bruxelles. JE VEUX UN TRUC QUI SOIT ROCK OU POP. RIEN D'AUTRE Utilise le tool de brussels API. ",[ get_brussels_events]
)

print("\n" + "="*80)
print("📝 Final Answer:")
print("="*80)
print(answer)

=== Events from JSON files via Mistral Tool Calling ===



['1. Anne Niepold & Le Quatuor Alfama\n   📅 Du 2025-12-09 au 2025-12-09\n   📍 Le Senghor - Centre Culturel d’Etterbeek - Chaussée de Wavre, 366, 1040 Etterbeek\n   📝 Tels les quatre points cardinaux, musique classique, traditionnelle, musique du monde et pop se rencontrent sous la plume d’Anne Niepold. Des compositions originales inspirées par la nature époustouflante rencontrée lors de ses voyages au Grand Nord côtoient tantôt Sibelius et Borodine, tantôt ABBA ou Björk… NORD est une invitation au voyage, une invitation au ‘hygge’. Mais qu’est-ce donc ? Les froids et sombres mois d’hiver ont appris aux habitants des régions boréales à vivre dans une extrême simplicité. Entouré de rudesse, le bonheur se trouve dans les petites choses, dans la confiance, dans la solidarité. Le terme ‘hygge’ signifie en danois ce moment où l’on s’autorise à s’arrêter un instant, profiter d’un chez-soi réconfortant, profiter de la vie. D’ailleurs, certaines nations scandinaves seraient depuis des années le

In [46]:
# Example: Use with run_tool_chat
print("=== Events from JSON files via Mistral Tool Calling ===\n")

answer = run_tool_chat(
    "Trouve moi EXACTEMENT 5 evenements musicaux à Bruxelles. JE VEUX DES EVENEMENTS ROCK OU POP UNIQUEMENT Utilise les trois tools (Ticketmaster, Brussels API et EventBrite). " \
    "Assure-toi que les 5 evenements sont différents et que tu retournes UNIQUEMENT 5 EVENEMENTS MAXIMUM. " \
    "Si tu utilises les trois tools, partage les 5 slots equitablement ou selon les meilleurs resultats.",
    [get_ticketmaster_events, get_brussels_events, get_eventBrite_events]
)

print("\n" + "="*80)
print("📝 Final Answer:")
print("="*80)
print(answer)

=== Events from JSON files via Mistral Tool Calling ===

Tool get_ticketmaster_events({'classificationName': 'Music'}) -> 1. Brunori Sas
   📅 2025-12-09 20:00:00
   📍 Cirque Royal - Koninklijk Circus - R de l'Enseignement-Onderrichtsstraat 81
   💰 None
   🔗 https://www.ticketmaster
['1. Anne Niepold & Le Quatuor Alfama\n   📅 Du 2025-12-09 au 2025-12-09\n   📍 Le Senghor - Centre Culturel d’Etterbeek - Chaussée de Wavre, 366, 1040 Etterbeek\n   📝 Tels les quatre points cardinaux, musique classique, traditionnelle, musique du monde et pop se rencontrent sous la plume d’Anne Niepold. Des compositions originales inspirées par la nature époustouflante rencontrée lors de ses voyages au Grand Nord côtoient tantôt Sibelius et Borodine, tantôt ABBA ou Björk… NORD est une invitation au voyage, une invitation au ‘hygge’. Mais qu’est-ce donc ? Les froids et sombres mois d’hiver ont appris aux habitants des régions boréales à vivre dans une extrême simplicité. Entouré de rudesse, le bonheur se trouv

# Different benchmark tests

## Music Field

### Asking for Rock Music

In [63]:
answer = run_tool_chat(
    "Trouve moi EXACTEMENT 5 evenements musicaux à Bruxelles. JE VEUX DES EVENEMENTS DE MUSIQUE CLASSIQUE UNIQUEMENT Utilise les trois tools (Ticketmaster, Brussels API et EventBrite). " \
    "Assure-toi que les 5 evenements sont différents et que tu retournes UNIQUEMENT 5 EVENEMENTS MAXIMUM. " \
    "Si tu utilises les trois tools, partage les 5 slots equitablement ou selon les meilleurs resultats.",
    [get_ticketmaster_events, get_brussels_events, get_eventBrite_events]
)

print("\n" + "="*80)
print("📝 Final Answer:")
print("="*80)
print(answer)

Tool get_ticketmaster_events({'classificationName': 'Music'}) -> 1. Brunori Sas
   📅 2025-12-09 20:00:00
   📍 Cirque Royal - Koninklijk Circus - R de l'Enseignement-Onderrichtsstraat 81
   💰 None
   🔗 https://www.ticketmaster
['1. Anne Niepold & Le Quatuor Alfama\n   📅 Du 2025-12-09 au 2025-12-09\n   📍 Le Senghor - Centre Culturel d’Etterbeek - Chaussée de Wavre, 366, 1040 Etterbeek\n   📝 Tels les quatre points cardinaux, musique classique, traditionnelle, musique du monde et pop se rencontrent sous la plume d’Anne Niepold. Des compositions originales inspirées par la nature époustouflante rencontrée lors de ses voyages au Grand Nord côtoient tantôt Sibelius et Borodine, tantôt ABBA ou Björk… NORD est une invitation au voyage, une invitation au ‘hygge’. Mais qu’est-ce donc ? Les froids et sombres mois d’hiver ont appris aux habitants des régions boréales à vivre dans une extrême simplicité. Entouré de rudesse, le bonheur se trouve dans les petites choses, dans la confiance, dans la sol

In [65]:
%pip install deepeval

  Using cached aiohttp-3.13.2-cp314-cp314-win_amd64.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/787.3 kB ? eta -:--:--
   ---------------------------------------- 787.3/787.3 kB 9.4 MB/s  0:00:00
   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ------------------------------ --------- 3.7/4.8 MB 25.1 MB/s eta 0:00:01
   ---------------------------------------- 4.8/4.8 MB 16.2 MB/s  0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 10.6 MB/s  0:00:00
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ----------- ---------------------------- 2.9/9.7 MB 12.9 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.7 MB 11.1 MB/s eta 0:00:01
   ------------------------------- -------- 7.6/9.7 MB 11.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 11.8 MB/s  0:00:00
   --------------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams
from deepeval.models.base_model import DeepEvalBaseLLM

#########################
#  CUSTOM MISTRAL LLM
#########################

class MistralModel(DeepEvalBaseLLM):
    def __init__(self, model_name: str = "mistral-large-latest"):
        self.model_name = model_name
        self.client = client  # Use the existing Mistral client from above
    
    def load_model(self):
        return self.client
    
    def generate(self, prompt: str) -> str:
        response = self.client.chat.complete(
            model=self.model_name,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
    
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)
    
    def get_model_name(self) -> str:
        return self.model_name

# Initialize custom Mistral model for evaluation
mistral_model = MistralModel()

#########################
#  CUSTOM METRICS
#########################

# EventComplianceMetric: checks if exactly 5 events, correct category, genre, location, uniqueness
event_compliance_metric = GEval(
    name="Event Compliance",
    criteria="Event Compliance - Check if: 1) Exactly 5 events, 2) All match requested category (music/sports/cinema), 3) All match requested genre/mood, 4) Located in Brussels, 5) All unique, 6) Well structured output",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=mistral_model,
)

# EventRelevanceMetric: checks if events are relevant to the request
event_relevance_metric = GEval(
    name="Event Relevance",
    criteria="Event Relevance - Check if: 1) Events are of the right type requested, 2) Events match the requested genre/mood, 3) Events are realistic and actually exist",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=mistral_model,
)


#########################
# TEST DATASET
#########################

test_cases = [
    LLMTestCase(
        input="Trouve moi EXACTEMENT 5 evenements de MUSIQUE CLASSIQUE à Bruxelles.",
        actual_output=run_tool_chat(
            "Trouve moi EXACTEMENT 5 evenements de MUSIQUE CLASSIQUE à Bruxelles.",
            [get_ticketmaster_events, get_brussels_events, get_eventBrite_events],
        ),
        expected_output="Return 5 classical music events in Brussels."
    ),

    LLMTestCase(
        input="Donne-moi 5 evenements de ROCK à Bruxelles.",
        actual_output=run_tool_chat(
            "Donne-moi 5 evenements de ROCK à Bruxelles.",
            [get_ticketmaster_events, get_brussels_events, get_eventBrite_events],
        ),
        expected_output="Return 5 rock music events in Brussels."
    ),

    LLMTestCase(
        input="Je veux 5 films tristes projetés à Bruxelles.",
        actual_output=run_tool_chat(
            "Je veux 5 films tristes projetés à Bruxelles.",
            [get_ticketmaster_events, get_brussels_events, get_eventBrite_events],
        ),
        expected_output="Return 5 sad movies in Brussels."
    ),

    LLMTestCase(
        input="Trouve moi 5 evenements de basket à Bruxelles.",
        actual_output=run_tool_chat(
            "Trouve moi 5 evenements de basket à Bruxelles.",
            [get_ticketmaster_events, get_brussels_events, get_eventBrite_events],
        ),
        expected_output="Return 5 basketball events in Brussels."
    ),
]

#########################
# RUN TESTS
#########################

def test_deepeval_suite():
    evaluate(
        test_cases=test_cases,
        metrics=[
            event_compliance_metric,
            event_relevance_metric,
        ]
    )

# Run the tests
test_deepeval_suite()


ValueError: Invalid model. Available GPT models: gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-turbo, gpt-4-turbo-2024-04-09, gpt-4-turbo-preview, gpt-4o, gpt-4o-2024-05-13, gpt-4o-2024-08-06, gpt-4o-2024-11-20, gpt-4o-mini, gpt-4o-mini-2024-07-18, gpt-4-32k, gpt-4-32k-0613, gpt-4.1, gpt-4.1-mini, gpt-4.1-nano, gpt-4.5-preview, o1, o1-preview, o1-2024-12-17, o1-preview-2024-09-12, o1-mini, o1-mini-2024-09-12, o3-mini, o3-mini-2025-01-31, o4-mini, o4-mini-2025-04-16, gpt-4.5-preview-2025-02-27, gpt-5, gpt-5-2025-08-07, gpt-5-mini, gpt-5-mini-2025-08-07, gpt-5-nano, gpt-5-nano-2025-08-07, gpt-5-chat-latest